In [425]:
class Nim:
    def __init__(self, heaps, player1 = 'Player 1', player2 = 'Player 1'):
        self.heaps = heaps
        self.names = (player1, player2)
        self.turn = 1
        
    def show(self):
        print('State of the game:', self.heaps)
        print('It is ', self.names[self.turn-1],'\'s turn. (Player ',self.turn, ')', sep='')
    
    def move(self,heap, num, player):  
        if self.turn == player:
            #print(self.heaps[heap-1]-num)
            if self.heaps[heap-1] - num >= 0 and num > 0:
                self.turn = (2-player)+1
                self.heaps[heap-1] = self.heaps[heap-1] - num
                
                ## check if game is over
                s = 0
                for i in self.heaps: s += i
                if s == 0: print("You," , self.names[player-1], " won the game!")
            else:
                print('You cannot take that many chips.')    
        else: print('It is not your turn.')
        self.show()
            
    def gamevalue(self):
        
        def dec_to_bin(x):
            '''Find the binary expansion of a number x, output as a list.'''
            pwr_of_two = 0
            expansion = []
            while 2**pwr_of_two <= x: # fit as many pwrs of 2 into x as possible
                pwr_of_two += 1
            pwr_of_two -= 1 # step down last count 
            while x > 0: # break down x into powers of 2 and save results in expansion
                if x - 2**pwr_of_two >= 0:
                    expansion.append(1)
                    x = x - 2**pwr_of_two
                    pwr_of_two = pwr_of_two - 1
                else:
                    expansion.append(0)
                    pwr_of_two = pwr_of_two - 1

            pwr_of_two += 1

            while pwr_of_two > 0: 
                expansion.append(0)
                pwr_of_two = pwr_of_two - 1

            return expansion
        
        def bin_to_dec(expansion): 
            '''recover the decimal from the binary expansion'''
            s = 0
            for k in range(len(expansion)): 
                s+=2**(len(expansion)-k-1)*expansion[k]
            return s
        
        def add_no_carry(expansions):
            total = [0] * len(expansions[0])
            for index, expansion in enumerate(expansions):
                total = [e1+e2 for e1, e2 in zip(total, expansion)]

            for index, element in enumerate(total):
                total[index] = element%2

            return total
        
        heaps = self.heaps
        
        # create binary expansions, then...
        # find length of longest binary expansion in list
        expansions = []
        maxexplen = 0
        for heap in heaps:
            expansions = [*expansions, dec_to_bin(heap)]
            maxexplen = max(maxexplen, len(expansions[-1])) 

        # Make the expansions compatible
        for index, expansion in enumerate(expansions):
            zeros = []
            for j in range(maxexplen - len(expansion)): 
                zeros.append(0)
                expansions[index] = [*zeros, *expansion]

        gv_expansion = add_no_carry(expansions)
        gv = bin_to_dec(gv_expansion)
        
        print('Game value:'.ljust(35), gv, '\nGame value in binary expansion: '.ljust(35), gv_expansion, sep='')
        
        # Recommend a winning move if the game is in a winning state
        if gv != 0:
            # i: index first one in game value
            # j: index of heap to make move in
            for i, pwr_2 in enumerate(gv_expansion):
                if pwr_2 == 1:
                    for j, heap in enumerate(expansions):
                        if heap[i] == 1: break
                    break
                    
            sum = 0
            l = len(gv_expansion)
            for k in range(i, l):
                    if gv_expansion[k] == 1 and expansions[j][k] == 1 : 
                        sum = sum - 2**(l-1-k)
                        #print(-2**(l-1-k))
                    if gv_expansion[k] == 1 and expansions[j][k] == 0 : 
                        sum = sum + 2**(l-1-k)
                        #print(-2**(l-1-k))                        

            print("Move in heap", j+1, "and remove", -sum, "pieces.")
        
        return [gv, gv_expansion, expansions]